## Homework 3
1. Input BGR images from webcam.
2. Detect your face, mouth, and eyes. 
3. Input BGRA images from files "mustache.png" and "hat.png" (hint: cv2.imread("mustache.png", cv2.IMREAD_UNCHANGED) to read 4 channels)
4. Perform <b> Alpha Blending </b> to add mustache and hat on the right position and orientation of your face.
5. The overlaid mustache and hat should be translated, rotated , and scaled according to the movement of your face. 
6. Show your output images.
7. Any idea on how to detect face features better? Try it and compare the results. (hint: modules of <i>dlib</i> or <i>mediapipe</i>)
8. (5pts bonus) Open/Close your mouth to toggle mustache on/off.
9. (5pts bonus) Blink your eye to toggle hat on/off.
10. Upload your Jupyter code file (*.ipynb)